**Hill‐O‐Beans Coffee Company blends four component beans into three final blends of coffee: one is
sold to luxury hotels, another to restaurants, and the third to supermarkets for store‐label brand.
The company has four reliable bean supplies: Argentine Abundo, Peruvian Colmado, Braxilian
Maximo, and Chilean Saboro. The table below summarizes the very precise recipe for the final coffee
blends, the cost and availability information for the four components, and the wholesale price per
pound of the final blends. The percentage indicates the fraction of each component to be used in
each blend.**

Component(lb) | Hotel | Restaurant | Market | Cost per pound | Max weekly availability

Abundo    |    20%   |  35%  |  10%   |   0.60      |       40000

Colmado | 40% |15% | 35% | 0.80 | 25000

Maximo | 15%| 20% | 40% | 0.55 | 20000

Saboro | 25%| 30% | 15% | 0.70 | 45000

Wholesale price per pound | 1.25 | 1.50 | 1.40


**The processor’s plant can handle no more than 100,000 pounds per week, and Hill‐O‐Beans would
like to operate at capacity. There is no problem in selling the final blends, although the Marketing
Department requires minimum production levels of 10,000, 25,000, and 30,000 pounds, respectively,
for the hotel, restaurant, and market blends.**

(a) In order to maximize weekly profit, how many pounds of each component should be purchased?
How many pounds of each blend should be produced?

In [1]:
from pulp import *
probA=LpProblem("ProblemA", LpMaximize)

bean = ["Abundo","Colmado","Maximo","Saboro"]
blend = ["Hotel","Restaurant","Market"]

plant_capacity = 100000

bean_cost = [0.6,0.8,0.55,0.7]
bean_aval = [40000,25000,20000,45000]
prod_level = [10000,25000,30000]
blend_price = [1.25,1.5,1.4]

perc = [[0.2,0.35,0.1],\
        [0.4,0.15,0.35],\
        [0.15,0.2,0.4],\
        [0.25,0.3,0.15]]

bean_cost = makeDict([bean],bean_cost)
bean_aval = makeDict([bean],bean_aval)
prod_level = makeDict([blend],prod_level)
blend_price = makeDict([blend],blend_price)
perc = makeDict([bean,blend],perc)

bean_vars = LpVariable.dicts('bean_vars',bean,lowBound=0,cat="Continuous")
blend_vars = LpVariable.dicts('blend_vars',blend,lowBound=0,cat="Continuous")

# Objective Function
probA += lpSum([blend_vars[bl]*blend_price[bl] for bl in blend]) - \
         lpSum([bean_vars[b]*bean_cost[b] for b in bean])

# Constraints
for b in bean:
    probA += bean_vars[b] <= bean_aval[b]
    probA += lpSum([perc[b][bl]*blend_vars[bl] for bl in blend])== bean_vars[b]

for bl in blend:
    probA += blend_vars[bl] >= prod_level[bl]

probA += lpSum([blend_vars[bl] for bl in blend]) <= plant_capacity

In [2]:
probA.writeLP("ProblemA.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


In [3]:
for v in probA.variables():
    print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print('Total cost=', value(probA.objective))

bean_vars_Abundo = 16375.0 	Reduced Cost = 0.0
bean_vars_Colmado = 19375.0 	Reduced Cost = 1.110223e-16
bean_vars_Maximo = 20000.0 	Reduced Cost = 0.0
bean_vars_Saboro = 16750.0 	Reduced Cost = 0.0
blend_vars_Hotel = 10000.0 	Reduced Cost = -5.5511151e-17
blend_vars_Market = 30000.0 	Reduced Cost = 0.0
blend_vars_Restaurant = 32500.0 	Reduced Cost = 2.4980018e-16
Total cost= 55200.0


(b) How much (per pound) should Hill‐O‐Beans be willing to pay for an additional pound of Maximo
beans in order to raise total profit?

In [6]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : bean_vars_Abundo <= 40000 	 -0.0 		 23625.0
_C2 : -bean_vars_Abundo + 0.2*blend_vars_Hotel + 0.1*blend_vars_Market + 0.35*blend_vars_Restaurant = -0.0 	 0.6 		 -1.8189894e-12
_C3 : bean_vars_Colmado <= 25000 	 -0.0 		 5625.0
_C4 : -bean_vars_Colmado + 0.4*blend_vars_Hotel + 0.35*blend_vars_Market + 0.15*blend_vars_Restaurant = -0.0 	 0.8 		 -1.8189894e-12
_C5 : bean_vars_Maximo <= 20000 	 4.25 		 -0.0
_C6 : -bean_vars_Maximo + 0.15*blend_vars_Hotel + 0.4*blend_vars_Market + 0.2*blend_vars_Restaurant = -0.0 	 4.8 		 -9.094947e-13
_C7 : bean_vars_Saboro <= 45000 	 -0.0 		 28250.0
_C8 : -bean_vars_Saboro + 0.25*blend_vars_Hotel + 0.15*blend_vars_Market + 0.3*blend_vars_Restaurant = -0.0 	 0.7 		 -0.0
_C9 : blend_vars_Hotel >= 10000 	 -0.085 		 -0.0
_C10 : blend_vars_Restaurant >= 25000 	 -0.0 		 -7500.0
_C11 : blend_vars_Market >= 30000 	 -0.965 		 -0.0
_C12 : blend_vars_Hotel + blend_vars_Market + blend_vars_Restaurant <= 100000 